Generative AI in Research: using Large Language Models (LLMs) to enhance and streamline the academic literature review process.

leverage RAG Techniques for summarizing papers, identifying connections across papers (authors, references, methods), uncovering key themes in them.

Download 2 papers (related to diffusion model) and convert them to .txt files in a directory named "data". Use these .txt files as input papers and evaluate if the RAG technique is giving good results.

In [1]:
!pip install pymupdf requests

### download any 2 papers of diffusion model and convert their pdf to .txt files

In [2]:
!rm -r data

In [3]:
import os
import requests
import fitz  # PyMuPDF

# Sample arXiv paper IDs related to diffusion models
arxiv_ids = [
    "2006.11239",  # Denoising Diffusion Probabilistic Models
    "2105.05233",  # Improved Denoising Diffusion Probabilistic Models
]

def download_pdf(arxiv_id, output_folder):
    url = f"https://arxiv.org/pdf/{arxiv_id}.pdf"
    pdf_path = os.path.join(output_folder, f"{arxiv_id}.pdf")
    response = requests.get(url)
    with open(pdf_path, "wb") as f:
        f.write(response.content)
    print(f"Downloaded {arxiv_id}")
    return pdf_path

def pdf_to_text(pdf_path, txt_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    with open(txt_path, "w", encoding="utf-8") as f:
        f.write(text)
    print(f"Converted to text: {txt_path}")

def main():
    data_dir = "data"
    os.makedirs(data_dir, exist_ok=True)

    for arxiv_id in arxiv_ids:
        pdf_path = download_pdf(arxiv_id, data_dir)
        txt_path = os.path.join(data_dir, f"{arxiv_id}.txt")
        pdf_to_text(pdf_path, txt_path)

if __name__ == "__main__":
    main()


Downloaded 2006.11239
Converted to text: data/2006.11239.txt
Downloaded 2105.05233
Converted to text: data/2105.05233.txt


In [4]:
!rm data/*.pdf

In [5]:
# Install dependencies
!pip install langchain langchain_community faiss-cpu sentence-transformers transformers networkx matplotlib spacy
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 76.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [6]:
import os, glob
import gc
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
import networkx as nx
import matplotlib.pyplot as plt
import spacy
import re
gc.collect()
import torch
torch.cuda.empty_cache()

In [7]:
def clean_text(text):
    import re
    # Remove inline citations like [14], [14, 27]
    text = re.sub(r"\[[0-9,\s]+\]", "", text)
    # Remove URLs
    text = re.sub(r"http\S+|www\.\S+", "", text)
    # Remove LaTeX math expressions
    text = re.sub(r"\$.*?\$", "", text)
    # Remove repeated words
    text = re.sub(r"\b(\w+)( \1\b)+", r"\1", text)
    # Remove special characters
    text = re.sub(r"[^a-zA-Z0-9.,;:?!\s]", "", text)
    # Remove excessive whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    return text


# Load a document and return its content
def load_document(file_path):
    loader = TextLoader(file_path)
    docs = loader.load()
    for doc in docs:
        doc.page_content = clean_text(doc.page_content)  # Apply cleaning here
    return docs

# Split the document into chunks ensuring each chunk is under the token limit
def split_document(docs, chunk_size=1500, chunk_overlap=50):
    # Make sure docs is always a list
    if not isinstance(docs, list):
        docs = [docs]

    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    return splitter.split_documents(docs)

# Vector store (use sentence embeddings)
def create_faiss_index(docs):
    embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
    return FAISS.from_documents(docs, embeddings)

# Load HuggingFace LLM
def load_llm():
    model_id = "google/flan-t5-xl"  # Better GPU utilization, faster than flan-t5-large
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_id).to("cuda")  # Move model to GPU
    #pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer, device=0)
    pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer, device=0, max_new_tokens=512)

    return HuggingFacePipeline(pipeline=pipe)


# Build RAG chain
def build_qa_chain(llm, vectorstore):
    return RetrievalQA.from_chain_type(llm=llm, retriever=vectorstore.as_retriever(), chain_type="stuff")

# Analyze a single document in chunks and store results with a chunk limit
def analyze_document(llm, docs, batch_size=1, max_chunks=10):
    results = {"Summary": [], "Connections": [], "Themes": []}
    chunks = split_document(docs)
    chunks = chunks[:max_chunks]

    for i in range(0, len(chunks), batch_size):
        batch = chunks[i:i + batch_size]

        for label, q in [
            ("Summary", "Summarize the following scientific paper text in concise bullet points. Include: Main contribution, Dataset used, method, Evaluation metrics and Key results."),
            ("Connections", "You are reading several research papers. Based on the passage below, what connections or similarities can you identify with other papers on diffusion models? Mention common techniques, models, datasets, or evaluation strategies."),
            ("Themes", "What are the central *research themes* in the following paper? List them as concise topics.")
        ]:
            prompts = [f"{q}\n\n---\n{chunk.page_content.strip()}" for chunk in batch]
            try:
                responses = llm.pipeline(prompts)
                for response in responses:
                    text = response['generated_text'].strip()
                    results[label].append(text)
                    print(f"\n🔍 {label}:\n{text}")
            except Exception as e:
                print(f"Error during {label} batch: {e}")
                results[label].extend(["Error"] * len(batch))

        del batch
        torch.cuda.empty_cache()
        gc.collect()

    return results


# Main pipeline
def process_all_documents(data_dir="data", max_chunks=10):
    files = glob.glob(os.path.join(data_dir, "*.txt"))
    results = {"Summary": [], "Connections": [], "Themes": []}

    llm = load_llm()

    for file_path in files:
        print(f"\n📄 Processing: {file_path}")
        # Load and clean the document
        doc = load_document(file_path)

        # Process the document in chunks
        doc_results = analyze_document(llm, docs=doc, max_chunks=max_chunks)

        # Collect results
        for label in results:
            results[label].extend(doc_results.get(label, []))

        # Free up GPU memory after processing each document
        del doc
        torch.cuda.empty_cache()
        gc.collect()  # Run garbage collection to free memory

    # Clean-up results (e.g., remove empty strings or redundant entries)
    for label in results:
        flat = [str(item).strip() for sublist in results[label] for item in (sublist if isinstance(sublist, list) else [sublist])]
        cleaned = [s for s in flat if s and s.lower() != "error"]
        combined_text = "\n".join(cleaned)

        if label == "Themes":
          final_theme = summarize_combined_output(llm, combined_text, label) # Remove duplicates
          themes = list(dict.fromkeys([line.strip() for line in final_theme.split("\n") if line.strip()]))
          results[label] = "\n".join(themes)
        else:
          results[label] = summarize_combined_output(llm, combined_text, label)


    return results

# Summarize combined chunk outputs into a single final output
def summarize_combined_output(llm, text, label):
    prompts = {
        "Summary": "You are a helpful scientific assistant. Based on the following combined summaries of a scientific paper, provide a single concise overall summary. Mention the main contribution, dataset used, method, evaluation metrics, and key results.",
        "Connections": "You are reading several research papers. Based on the following notes, summarize the common connections or similarities across papers, focusing on shared techniques, datasets, or models.",
        "Themes": "Summarize the central research themes mentioned in the combined text below. List them as concise, broad topics."
    }
    prompt = f"{prompts[label]}\n\n{text}"
    try:
        response = llm.pipeline(prompt)
        return response[0]["generated_text"].strip()
    except Exception as e:
        print(f"Error generating final {label}: {e}")
        return "Error"


# Main execution
results = process_all_documents(data_dir="data", max_chunks=10)  # Limit the number of chunks for faster processing
# Final outputs
final_connections = results.get("Connections", "")
final_themes = results.get("Themes", "")
final_summary = results.get("Summary", "")

# Display summaries
print("\nFinal Summary:")
print(final_summary)
print("\nFinal Connections Across Papers:")
print(final_connections)
print("\nFinal Themes:")
print(final_themes)



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0
<ipython-input-7-ae6dbeb04d27>:48: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  return HuggingFacePipeline(pipeline=pipe)



📄 Processing: data/2105.05233.txt

🔍 Summary:
We show that diffusion models can achieve image sample quality superior to the current stateoftheart generative models. We achieve this on unconditional im age synthesis by nding a better architecture through a series of ablations. For conditional image synthesis, we further improve sample quality with classier guidance: a simple, computeefcient method for trading off diversity for delity using gradients from a classier. We achieve an FID of 2.97 on ImageNet 128128, 4.59 on ImageNet 256256, and 7.72 on ImageNet 512512, and we match BigGANdeep even with as few as 25 forward passes per sample. Finally, we nd that classier guidance combines well with upsampling diffusion models, further improving FID to 3.94 on ImageNet 256256 and 3.85 on ImageNet 512512. We release our code at 1 Introduction

🔍 Connections:
ImageNet 512512 images, they are not yet capable of producing highquality images.

🔍 Themes:
Image Synthesis

🔍 Summary:
Main contributi

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset



🔍 Summary:
Main contribution: We introduce a new architecture for diffusion models that improves FID and enables a classier to guide the model during sampling. We show that our architecture achieves stateoftheart on unconditional image synthesis tasks and with classier guidance achieve stateoftheart on conditional image synthesis.

🔍 Connections:
We use a gradient from a classier to guide a diffusion model during sampling.

🔍 Themes:
We improve the architecture of diffusion models by introducing a new architecture that combines a classier with a gradient guide.

🔍 Summary:
--- with some noise where the signal to noise ratio is determined by the timestep t. For the remainder of this paper, we assume that the noise is drawn from a diagonal Gaussian distribution, which works well for natural images and simplies various derivations. 2 A diffusion model learns to produce a slightly more denoised xt1 from xt. To train these models, each sample in a minibatch is produced by randomly drawing 

Token indices sequence length is longer than the specified maximum sequence length for this model (603 > 512). Running this sequence through the model will result in indexing errors



🔍 Summary:
Main contribution: We introduce a new method for learning the posterior of a forward process from a pxt1xt model. The forward process is xed to a Markov chain that gradually adds Gaussian noise to the data according to a variance schedule 1, . . ., T: qx1:T x0 : T Y t1 qxtxt1, qxtxt1 : Nxt; p1 txt1, tI 2 Training is performed by optimizing the usual variational bound on negative log likelihood: E log px0 Eq log px0:T qx1:T x0 Eq log pxT X t1 log pxt1xt qxtxt1 : L 3 The forward process variances t can be learned by reparameterization or held constant as hyperparameters, and expressiveness of the reverse process is ensured in part by the choice of Gaussian conditionals in pxt1xt, because both processes have the same functional form when t is small . Equation 5 uses KL divergence to directly compare pxt1xt against forward process posteriors, which are tractable when conditioned on x0: qxt1xt, x0 Nxt1; txt, x0, tI, where txt, x0: t1t 1 t x0 t1 t1 1 t xt and t : 1 t1 1 t t

🔍 Co